In [7]:
%matplotlib qt

QUESTION F

In [8]:
#Ishanya 21329

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

# VTOL Parameter File

# Physical parameters of the VTOL known to the controller
mc = 1.0  # kg
mr = 0.2  # kg
Jc = 0.01  # kg m^2
d = 0.1  # m
mu = 0.1  # kg/s
g = 9.81  # m/s^2
F_wind = 0.0  # Wind disturbance force is zero initially

# Parameters for animation
length = 10.0

# Initial Conditions
z0 = 4.0  # Initial lateral position (m)
h0 = 0.0  # Initial altitude (m)
theta0 = 0.0  # Initial roll angle (rad)
zdot0 = 0.0  # Initial lateral velocity (m/s)
hdot0 = 0.0  # Initial climb rate (m/s)
thetadot0 = 0.0  # Initial roll rate (rad/s)
target0 = 5.5  # Initial target position (m)

# Simulation Parameters
t_start = 0.0  # Start time of simulation
t_end = 10.0  # End time of simulation
Ts = 0.01  # Sample time for simulation
t_plot = 0.1  # The plotting and animation is updated at this rate

# Saturation limits
fmax = 5.0  # Max Force, N

# Equilibrium force
Fe = (mc + 2 * mr) * g

# Mixing matrix
mixing = np.linalg.inv(np.array([[1.0, 1.0], [d, -d]]))


class VTOLAnimation:
    def __init__(self):
        self.flagInit = True
        self.fig, self.ax = plt.subplots()
        self.handle = []
        self.length = length
        plt.plot([0.0, length], [0.0, 0.0], 'k')
        plt.axis([-length / 5, length + length / 5, -length / 5, length + length / 5])

    def update(self, x, target=0.0):
        z = x.item(0)
        h = x.item(1)
        theta = x.item(2)

        self.drawVehicle(z, h, theta)
        self.drawTarget(target)

        if self.flagInit == True:
            self.flagInit = False

    def drawVehicle(self, z, h, theta):
        x1 = 0.3
        x2 = 0.9
        x3 = 1.2
        y1 = 0.15
        y2 = 0.03
        pts = np.array([
            [x1, y1],
            [x1, 0],
            [x2, 0],
            [x2, y2],
            [x3, y2],
            [x3, -y2],
            [x2, -y2],
            [x2, 0],
            [x1, 0],
            [x1, -y1],
            [-x1, -y1],
            [-x1, 0],
            [-x2, 0],
            [-x2, -y2],
            [-x3, -y2],
            [-x3, y2],
            [-x2, y2],
            [-x2, 0],
            [-x1, 0],
            [-x1, y1],
            [x1, y1]]).T
        R = np.array([
            [np.cos(theta), np.sin(theta)],
            [-np.sin(theta), np.cos(theta)],
        ])
        pts = R.T @ pts
        pts = pts + np.tile(np.array([[z], [h]]), (1, pts.shape[1]))
        xy = np.array(pts.T)

        if self.flagInit == True:
            self.handle.append(mpatches.Polygon(xy, facecolor='lightblue', edgecolor='black'))
            self.ax.add_patch(self.handle[0])
        else:
            self.handle[0].set_xy(xy)

    def drawTarget(self, target=0.0):
        w = 0.1
        h = 0.05
        pts = np.matrix([
            [target + w / 2.0, h],
            [target + w / 2.0, 0],
            [target - w / 2.0, 0],
            [target - w / 2.0, h],
            [target + w / 2.0, h]])

        if self.flagInit == True:
            self.handle.append(mpatches.Polygon(pts, facecolor='green', edgecolor='black'))
            self.ax.add_patch(self.handle[1])
        else:
            self.handle[1].set_xy(pts)


def main():
    animation = VTOLAnimation()
    t = np.arange(0.0, t_end / 4, Ts)
    x = np.array([z0, h0 + 0.1, theta0])
    animation.update(x, target0)
    plt.pause(10)

    for ti in t:
        x = np.array([z0, h0 + 0.1 + ti, theta0 + ti / 100])
        target = target0
        animation.update(x, target)
        plt.pause(0.01)

    plt.show()


main()
